# Анализ данных сервиса по чтению книг по подписке
Коронавирус застал мир врасплох, изменив привычный порядок вещей. На какое-то время жители городов перестали выходить на улицу, посещать кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.

В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

In [1]:
# импортируем библиотеки 
import pandas as pd 
from sqlalchemy import text, create_engine 

In [2]:
# устанавливаем параметры 
db_config = {'user': 'praktikum_student', # имя пользователя 
'pwd': 'Sdf4$2;d-d30pp', # пароль 
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
'port': 6432, # порт подключения 
'db': 'data-analyst-final-project-db'} # название базы данных 
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config) 

In [3]:
# сохраняем коннектор 
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
con=engine.connect()

Выведем для каждой таблицы количество записей, формат данных колонок и первые пять строк таблицы.

In [4]:
tables=['books', 'authors', 'publishers', 'ratings', 'reviews']
for i in range (0,len(tables)):
    display('Таблица '+tables[i])
    
    query = 'select count(*) from '+tables[i]
    display(pd.io.sql.read_sql(sql=text(query), con = con))
    
    query = 'SELECT column_name, column_default, data_type FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = \''+tables[i]+'\''
    display(pd.io.sql.read_sql(sql=text(query), con = con))
  
    query = 'select * from '+tables[i]+' limit 5'
    display(pd.io.sql.read_sql(sql=text(query), con = con))

'Таблица books'

,count
0,1000


,column_name,column_default,data_type
0,book_id,None,integer
1,author_id,None,integer
2,num_pages,None,integer
3,publication_date,None,date
4,publisher_id,None,integer
5,title,None,text


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


'Таблица authors'

...

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


'Таблица reviews'

,count
0,2793


,column_name,column_default,data_type
0,review_id,None,integer
1,book_id,None,integer
2,username,None,text
3,text,None,text


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Нам предоставлены 5 таблиц, ниже приведена информация о форматах столбцов таблицы, отмечены первичный и вторичные ключи.

__books__ - 1000 записей

    book_id	integer PK
    author_id	integer SK
    num_pages	integer
    publication_date	date
    publisher_id	integer SK
    title	text

__authors__ - 636 записей

    author_id	integer PK
    author	text

__publishers__ - 340 записей

    publisher_id	integer PK
    publisher	text

__ratings__ - 6456 записей

    rating_id	integer PK
    book_id	integer SK
    rating	integer
    username	text

__reviews__ - 2793 записи

    review_id	integer PK
    book_id	integer SK
    username	text
    text	text
    
По построению базы есть следующие соображения:

1. Таблица книг построена таким образом, что может включать в себя лишь книги с одним автором. Книги с несколькими авторами можно обработать только объединяя авторов в одну строку, но это некорректная обработка.
2. В таблице книг отсутствует информация о жанре, ISBN, языке издания
3. В таблице книг не хватает поля "под редакцией" для книг-сборников статей/рассказов различных авторов. Можно также добавить какое-то поле/отдельную таблицу для связки серий книг и многотомников.
4. Не хватает таблицы Users в которой бы были user_id и какие то данные о людях, осталяющих обзоры и оценки. Это бы позволило получать доп. данные и избегать путаницы при идентификации пользователя по не уникальному параметру username.
5. Нет данных о датах оставления обзоров и оценок. Если для оценок отсутствие допустимо, то для обзоров дата может содержать полезную информацию.

#### Посчитайте, сколько книг вышло после 1 января 2000 года;

In [5]:
query = 'select count(*) from books where publication_date >\'01.01.2000\'' 
answer1 = pd.io.sql.read_sql(sql=text(query), con = con)
display('После 01.01.2000 было выпущено '+str(answer1.iloc[0,0])+' книг')

'После 01.01.2000 было выпущено 819 книг'

После 01.01.2000 было выпущено 819 книг.

#### Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [6]:
query = 'select b.book_id, author_id, title, num_pages, publication_date, publisher_id, r.mean_rating, r2.review_num \
    from books as b \
    left join (select book_id, avg(rating) as mean_rating \
                from ratings \
                group by book_id) r on b.book_id=r.book_id \
    left join (select book_id, count(review_id) as review_num \
                from reviews \
                group by book_id ) r2 on b.book_id=r2.book_id '
answer2 = pd.io.sql.read_sql(sql=text(query), con = con)
answer2.head(10)

,book_id,author_id,title,num_pages,publication_date,publisher_id,mean_rating,review_num
0,652,3,The Body in the Library (Miss Marple #3),191,2006-09-01,50,4.500000,2.0
1,273,352,Galápagos,324,1999-01-12,88,4.500000,2.0
2,51,54,A Tree Grows in Brooklyn,496,2006-05-30,135,4.250000,5.0
3,951,540,Undaunted Courage: The Pioneering First Missio...,592,2003-10-06,269,4.000000,2.0
4,839,332,The Prophet,127,2010-01-01,251,4.285714,4.0
5,70,430,American Gods (American Gods #1),635,2005-03-04,150,4.000000,5.0
6,350,501,Influence: The Psychology of Persuasion,320,2006-12-26,127,4.333333,3.0
7,758,184,The Idiot,667,2003-04-08,183,4.250000,3.0
8,539,114,Rebecca,441,2007-12-01,317,4.000000,4.0
9,874,248,The Source,1080,2002-07-09,243,3.500000,2.0


Для каждой книги посчитали среднюю оценку mean_rating и количество обзоров review_num в таблице answer2

#### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [7]:
query='select p.publisher_id, publisher, t1.num_books \
from publishers p \
inner join (select publisher_id, count(book_id) as num_books \
            from books \
            where num_pages >50 \
            group by publisher_id) t1 on t1.publisher_id=p.publisher_id \
order by t1.num_books DESC'
answer3 = pd.io.sql.read_sql(sql=text(query), con = con)
answer3.head(3)

,publisher_id,publisher,num_books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25


Издательство, выпустившее наибольшее количество книг толще 50 страниц - Penguin Books, 42 издания свыше 50 страниц.

#### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [8]:
query='select a.author_id, a.author, t1.mean_rating \
from (select author_id, avg(r2.rating) as mean_rating \
        from books b \
        inner join ratings r2 on b.book_id =r2.book_id \
        where b.book_id in (select book_id \
                            from ratings r \
                            group by book_id \
                            having count(rating_id)>=50) \
        group by author_id) as t1 \
inner join author a on t1.author_id=a.author_id \
order by t1.mean_rating DESC'
answer4 = pd.io.sql.read_sql(sql=text(query), con = con)
answer4.head(3)

,author_id,author,mean_rating
0,236,J.K. Rowling/Mary GrandPré,4.287097
1,402,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,240,J.R.R. Tolkien,4.246914


Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré, оценка 4.29. Как раз видны недостатки архитектуры базы - двухголовый автор это вообще говоря неправильно.

#### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [9]:
query='with table1 as (select username, count(review_id) as num_rev \
from reviews \
where username in (select username \
                    from ratings \
                    group by username \
                    having count(rating_id)>48) \
group by username) \
select avg(num_rev) \
from table1'
answer5 = pd.io.sql.read_sql(sql=text(query), con = con)
answer5.head(10)

,avg
0,24.0


Пользователи поставившие 48 оценок в среднем написали 24 обзора.

#### Вывод

Нам предоставлен доступ к базе данных, содержащей 1000 записей книг, преимущественно англоязычных. Не предоставлена информация о статистике заказов, ценах, клиентах и т.п. для полноценного анализа.

По построению базы есть следующие соображения:

1. Таблица книг построена таким образом, что может включать в себя лишь книги с одним автором. Книги с несколькими авторами можно обработать только объединяя авторов в одну строку, но это некорректная обработка.
2. В таблице книг отсутствует информация о жанре, ISBN, языке издания
3. В таблице книг не хватает поля "под редакцией" для книг-сборников статей/рассказов различных авторов. Можно также добавить какое-то поле/отдельную таблицу для связки серий книг и многотомников.
4. Не хватает таблицы Users в которой бы были user_id и какие то данные о людях, осталяющих обзоры и оценки. Это бы позволило получать доп. данные и избегать путаницы при идентификации пользователя по не уникальному параметру username.
5. Нет данных о датах оставления обзоров и оценок. Если для оценок отсутствие допустимо, то для обзоров дата может содержать полезную информацию.

На основании базы данных построены выборки, позволившие установить, что:

1. 819 книг из 1000 выпущены после 2000 года. Т.е. это какие-то современные книги.
2. Построена сводная таблица с указанием количества обзоров и средней оценкой для каждой книги.
3. Подготовлен список издательств по количеству публикаций в базе
4. Подготовлен рейтинг авторов по средней оценке книг.
5. Установлено, что активные пользователи в среднем оставляют 24 обзора на книги.